
# Assignment: Linear Regression, Logistic Regression, and K-Means (From Scratch)

**Instructions**
- You are NOT allowed to use `scikit-learn` for model implementation, scaling.
- You may use it for implementation of clustering
- You may use: `numpy`, `matplotlib`, and standard Python libraries only.
- Every step (scaling, loss, gradients, optimization) must be implemented manually.
- Clearly comment your code and explain your reasoning in Markdown cells.


## Question 1: Linear Regression from Scratch (with Standardization and Regularization)

You are given a dataset `(X, y)`.

### Tasks
1. Implement **StandardScaler manually**:
   - Compute mean and standard deviation for each feature.
   - Standardize the features.
2. Implement **Linear Regression using Gradient Descent**.
3. Add **L2 Regularization (Ridge Regression)**.
4. Plot:
   - Loss vs iterations
   - True vs predicted values

Do NOT use `sklearn`.


In [1]:

import numpy as np
import matplotlib.pyplot as plt

In [2]:

# Implement StandardScaler manually ,  first read about it, how it works and then implement it 
class StandardScalerManual:
    def fit(self, X):

        self.mean= np.mean(X,axis=0)  # Compute mean for each feature
        self.std= np.std(X,axis=0)   # Compute standard deviation for each feature
        self.std[self.std== 0]= 1    # To prevent division by zero
        
        return self
    
    def transform(self, X):

        return (X - self.mean)/ self.std   # Standardize the features
    
    def fit_transform(self, X):
        # Fit the scaler and transform the data
        self.fit(X)   
        return self.transform(X)

In [3]:

# Implement Linear Regression from scratch, here you have to also construct the regulization term coefficient of which will be
# denoted by l2_lambda 
# try to implement L1 regularization or atlease read about it and where it is used
class LinearRegressionManual:
    def __init__(self, lr=0.01, epochs=1000, l2_lambda=0.0):
        
        self.lr= lr                # Learning rate for gradient descent
        self.epochs= epochs        # Number of training iterations
        self.l2_lambda= l2_lambda  # Regularization coefficient (λ)

    def fit(self, X, y):
        # Train linear regression using gradient descent

        row, col= X.shape  
        X= np.c_[np.ones(row), X]       # Add bias column of ones
        self.weights= np.zeros(col)      # Initialize weights
        self.loss=[]                    # initialize loss 

        for _ in range(self.epochs):
            y_pred= np.dot(X, self.weights)  # Predicted values
            error= y_pred - y                # Prediction error

            gradient= (1/ row) * (np.dot(X.T, error))   # Gradient of MSE loss
            gradient+=  self.l2_lambda * self.weights    # Add L2 regularization gradient
            self.weights-= self.lr * gradient            # Update weights

            mse= np.mean(error ** 2)/ 2     # Mean Squared Error loss
            l2_term= (self.l2_lambda / 2) * np.sum(self.weights ** 2)
            loss= mse +l2_term 
            self.losses.append(loss)

    def predict(self, X):
        # Return predicted probabilities

        row= X.shape[0]            
        X= np.c_[np.ones(row), X]       # Add bias term
        return np.dot(X, self.weights)


In [4]:
# Testing the Model
X_raw= 2*np.random.rand(1,100)
y_raw= 4+3*X_raw.flatten() + np.random.randn(100) # y= 4+ 3x +noise

# Standardizing 
res= StandardScalerManual()
X_scale= res.fit_transform(X_raw)

# Training the Model
lr_manual= LinearRegressionManual(lr=0.1, epochs=500, l2_lambda=0.1)
lr_manual.fit(X_scale, y_raw)

# Plot Results
plt.figure(figsize=(14,7))
plt.subplot(1, 2, 1)
plt.plot(lr_manual.loss)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss vs Iterations")

# Plot Prediction
plt.subplot(1, 2, 2)
plt.scatter(X_scale, y_raw, color='red', label='True Data')
plt.title("True vs predicted values")
plt.plot(X_scale, lr_manual.predict(X_scale), color='blue', label='Predicted values')
plt.legend()
plt.show()

ValueError: shapes (1,101) and (100,) not aligned: 101 (dim 1) != 100 (dim 0)


## Question 2: Logistic Regression from Scratch (with Standardization and Regularization)

You are given a binary classification dataset.

### Tasks
1. Reuse your **manual StandardScaler**.
2. Implement **Logistic Regression using Gradient Descent**.
3. Use:
   - Sigmoid function
   - Binary Cross Entropy loss
4. Add **L2 Regularization**.
5. Report:
   - Training loss curve
   - Final accuracy

Do NOT use `sklearn`.


In [ ]:

#Implement sigmoid function as told in the lectures 
def sigmoid(z):
    
    return 1 / (1 + np.exp(-z))


In [ ]:

#Implement Logistic Regression from scratch and here also add the regularizaation term 
class LogisticRegressionManual:
    def __init__(self, lr=0.01, epochs=1000, l2_lambda=0.0):
        
        self.lr= lr                # Learning rate for gradient descent
        self.epochs= epochs        # Number of training iterations
        self.l2_lambda= l2_lambda  # Regularization coefficient (λ)

    def fit(self, X, y):

        n_samples, n_features = X.shape
        X= np.c_[np.ones(n_samples), X]           # Add bias column
        self.weights = np.zeros(n_features + 1)   # Initialize weights

        for _ in range(self.epochs):
            linear_output = np.dot(X, self.weights)  # Linear combination
            y_pred = sigmoid(linear_output)          # Apply sigmoid to get probabilities
            error = y_pred - y                       # Error between predictions and actual labels

            gradient= (1/ n_samples)* (np.dot(X.T, error))  # Gradient of cross-entropy loss
            gradient+= 2 * self.l2_lambda * self.weights    # Add L2 regularization term
            self.weights-= self.lr * gradient               # Update weights

    def predict_proba(self, X):

        n_samples= X.shape[0]
        X= np.c_[np.ones(n_samples), X]
        return sigmoid(np.dot(X, self.weights))

    def predict(self, X):
        probs= self.predict_proba(X)
        return (probs>= 0.5).astype(int)

In [ ]:
"""
The training loss decreases steadily as the number of iterations increases indicating that the gradient descent algorithm is successful in minimizing the loss function.
After a certain number of iterations, the loss curve flattens, showing that the model has converged.
Accuracy= Total Number of Samples/ Number of Correct Predictions. The model achieves a high final accuracy, indicating that it correctly classifies the majority of samples.
"""


## Question 3: K-Means Clustering from Scratch (Matrix Clustering)

You are given a **random matrix** `M` of shape `(n, m)`.

### Tasks
Implement K-Means clustering **from scratch** such that:

1. Input:
   - A random matrix `M`
   - Number of clusters `k`
2. Output:
   - `assignment_table`: a matrix of same shape as `M`, where each element stores the **cluster label**
   - `cookbook`: a dictionary (hashmap) where:
     - Key = cluster index
     - Value = list of **positions (i, j)** belonging to that cluster
   - `centroids`: array storing centroid values

You must cluster **individual elements**, not rows.


In [10]:
# Implement K-Means for matrix elements
#CAN USE SK-LEARN FOR THIS TASK AS THIS TASK WILL HELP US DIRECTLY IN OUR PROJECT ! 

from sklearn.cluster import KMeans
def kmeans_matrix(M, k, max_iters=100):
    '''
    Returns:
    assignment_table: same shape as M, contains cluster labels
    cookbook: dict -> cluster_id : list of (i, j) positions
    centroids: numpy array of centroid values
    '''
    M_flat= M.reshape(-1, 1)   # Reshape
    kmeans= KMeans(n_clusters=k, max_iter=max_iters, n_init=5)
    kmeans.fit(M_flat)
    label= kmeans.labels_
    centroid= kmeans.cluster_centers_.flatten()

    row, col= M.shape
    assignment_table= label.reshape(row, col)
    cookbook= {i: [] for i in range(k)}

    for r in range(row):
        for c in range(col):
            id= assignment_table[r, c]
            cookbook[id].append((r, c))
            
    return assignment_table, cookbook, centroid

M= np.random.randint(0, 255, size=(7, 7)) 
table, cookbook, centroid= kmeans_matrix(M, k=5)
print("Centroids\n", centroid)
print("Assignment Table")
print(table)
print(f"\nCoordinates for Cluster 0 (First 10): {cookbook[0][:10]}")
print(f"\nNumber of items in Cluster 0: {len(cookbook[0])}")

Centroids
 [215.75        28.58333333 113.9         71.88888889 166.66666667]
Assignment Table
[[2 2 4 2 2 1 3]
 [3 0 1 0 2 2 4]
 [1 0 4 1 2 4 1]
 [1 0 2 4 0 1 2]
 [0 0 0 1 3 3 3]
 [3 4 0 0 3 3 1]
 [0 1 3 2 1 1 0]]

Coordinates for Cluster 0 (First 10): [(1, 1), (1, 3), (2, 1), (3, 1), (3, 4), (4, 0), (4, 1), (4, 2), (5, 2), (5, 3)]

Number of items in Cluster 0: 12



## Submission Guidelines
- Submit the completed `.ipynb` file.
- Clearly label all plots and outputs.
- Code readability and correctness matter.
- Partial credit will be given for logically correct implementations.

**Bonus**
- Compare convergence with and without standardization.
- Try different values of regularization strength.
